In [47]:
#!pip install pandas
#!pip install zipfile
#!pip install kaggle

# Import libraries
import pandas as pd
import zipfile
import kaggle

# Download dataset from kaggle using the Kaggle API
!kaggle datasets download -d sobhanmoosavi/us-accidents

In [3]:
# Extract the file from the downloaded zip file
zipfile_name = 'us-accidents.zip'
with zipfile.ZipFile(zipfile_name, 'r') as file:
    file.extractall()

In [ ]:
# Read in the csv file
raw_data = pd.read_csv("../data/raw/us-accidents.csv")

In [49]:
# Check columns and data types
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2845342 entries, 0 to 2845341
Data columns (total 47 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   ID                     object 
 1   Severity               int64  
 2   Start_Time             object 
 3   End_Time               object 
 4   Start_Lat              float64
 5   Start_Lng              float64
 6   End_Lat                float64
 7   End_Lng                float64
 8   Distance(mi)           float64
 9   Description            object 
 10  Number                 float64
 11  Street                 object 
 12  Side                   object 
 13  City                   object 
 14  County                 object 
 15  State                  object 
 16  Zipcode                object 
 17  Country                object 
 18  Timezone               object 
 19  Airport_Code           object 
 20  Weather_Timestamp      object 
 21  Temperature(F)         float64
 22  Wind_Chill(F)     

In [81]:
# Create new DF with only selected columns from existing DF
df = raw_data.iloc[:,[1,2,4,5,15,21,27,29]]
df.head()

,Severity,Start_Time,Start_Lat,Start_Lng,State,Temperature(F),Wind_Speed(mph),Weather_Condition
0,3,2016-02-08 00:37:08,40.108910,-83.092860,OH,42.1,10.4,Light Rain
1,2,2016-02-08 05:56:20,39.865420,-84.062800,OH,36.9,NaN,Light Rain
2,2,2016-02-08 06:15:39,39.102660,-84.524680,OH,36.0,NaN,Overcast
3,2,2016-02-08 06:51:45,41.062130,-81.537840,OH,39.0,NaN,Overcast
4,3,2016-02-08 07:53:43,39.172393,-84.492792,OH,37.0,10.4,Light Rain


In [ ]:
# Convert date object to datetime type
df["Start_Time"] = pd.to_datetime(df['Start_Time'])

In [83]:
# Create a new Hour column based on Time
df["Hour"] = df["Start_Time"].dt.hour
df

/var/folders/zc/nktjdzgx11d7s4b_g548x88c0000gp/T/ipykernel_54752/1531486614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Hour"] = df["Start_Time"].dt.hour


,Severity,Start_Time,Start_Lat,Start_Lng,State,Temperature(F),Wind_Speed(mph),Weather_Condition,Hour
0,3,2016-02-08 00:37:08,40.108910,-83.092860,OH,42.1,10.4,Light Rain,0
1,2,2016-02-08 05:56:20,39.865420,-84.062800,OH,36.9,NaN,Light Rain,5
2,2,2016-02-08 06:15:39,39.102660,-84.524680,OH,36.0,NaN,Overcast,6
3,2,2016-02-08 06:51:45,41.062130,-81.537840,OH,39.0,NaN,Overcast,6
4,3,2016-02-08 07:53:43,39.172393,-84.492792,OH,37.0,10.4,Light Rain,7
...,...,...,...,...,...,...,...,...,...
2845337,2,2019-08-23 18:03:25,34.002480,-117.379360,CA,86.0,13.0,Fair,18
2845338,2,2019-08-23 19:11:30,32.766960,-117.148060,CA,70.0,6.0,Fair,19
2845339,2,2019-08-23 19:00:21,33.775450,-117.847790,CA,73.0,10.0,Partly Cloudy,19
2845340,2,2019-08-23 19:00:21,33.992460,-118.403020,CA,71.0,8.0,Fair,19


In [84]:
# Remove rows with missing values
df = df.dropna()

In [85]:
# Chexk if all missing values have been removed
df.isna().sum()

Severity             0
Start_Time           0
Start_Lat            0
Start_Lng            0
State                0
Temperature(F)       0
Wind_Speed(mph)      0
Weather_Condition    0
Hour                 0
dtype: int64

In [ ]:
# Create Period of Day column based on Time
df['Period'] = (df['Start_Time'].dt.hour % 24 + 4) // 4
df['Period'].replace({1: 'Late Night',
                      2: 'Early Morning',
                      3: 'Morning',
                      4: 'Noon',
                      5: 'Evening',
                      6: 'Night'}, inplace=True)

In [80]:
# Download cleaned dataset as csv file
df.to_csv("../data/processed/us-accidents.csv", index=False)